In [1]:
from pprint import pprint
from datetime import datetime

import ray
import numpy as np
import pandas as pd

In [2]:
ray.init(num_cpus=16)

2023-04-07 14:15:21,866	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.11.0
Ray version:,2.3.0
Dashboard:,http://127.0.0.1:8265


In [3]:
AMNT_OF_CORES_TO_UTIL = 8
AMNT_OF_NDARRAYS = 2 * AMNT_OF_CORES_TO_UTIL
DIM_OF_SQUARE = 14000
DIM_OF_NPARRAY = (DIM_OF_SQUARE, DIM_OF_SQUARE)
ITERATIONS = [ 17]

In [4]:
def get_random_sqr_matrx(dim = DIM_OF_SQUARE):
    return np.random.rand(dim, dim)

def add_sqr_matrx():
    mtrx1 = get_random_sqr_matrx()
    mtrx2 = get_random_sqr_matrx()
    assert mtrx1.shape == mtrx2.shape, f'Matrices does not have equal shape'
    return np.add(mtrx1, mtrx2)

@ray.remote
def add_sqr_ray_matrx():
    mtrx1 = get_random_sqr_matrx()
    mtrx2 = get_random_sqr_matrx()
    assert mtrx1.shape == mtrx2.shape, f'Matrices does not have equal shape'
    return np.add(mtrx1, mtrx2)

In [5]:
def no_ray(iterations: int = 3):
    for i in range(iterations):
        add_sqr_matrx()

def with_ray(iterations: int = 3):
    futures = [add_sqr_ray_matrx.remote() for _ in range(iterations)]
    ray.get(futures)

def with_ray_return(iterations: int = 3):
    futures = [add_sqr_ray_matrx.remote() for _ in range(iterations)]
    return ray.get(futures)

In [6]:
def time_func(func, *args, **kwargs):
    start = datetime.now()
    func(*args, **kwargs)
    return datetime.now() - start

In [ ]:
funcs = [no_ray, with_ray, with_ray_return]
comparison = [[time_func(func, i) for i in ITERATIONS] for func in funcs]

In [ ]:
funcs_names = [func.__name__ for func in funcs]
df = pd.DataFrame(
    data=np.array(comparison).T,
    columns=funcs_names
)

In [ ]:
print(df)